In [46]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import gmaps
# Import API key
from api_keys import gkey
import json

# Incorporated citipy to determine city based on latitude and longitude
CityDataFrame = pd.read_csv('weather_dataframe.csv') 

In [2]:
CityDataFrame.reset_index()  
CityDataFrame = CityDataFrame.drop(columns={'Unnamed: 0'}) 
CityDataFrame.head()

,city name,Lat,Lng,MaxTemp,Humidity,Cloudiness,WindSpeed,Date,country
0,ixtapa,20.7000,-105.2000,78.80,53,1,4.61,1612032599,MX
1,hasaki,35.7333,140.8333,28.40,68,20,3.44,1612032599,JP
2,jiazi,22.8779,116.0670,55.51,88,3,11.56,1612032599,CN
3,atuona,-9.8000,-139.0333,78.78,78,19,16.11,1612032600,PF
4,berdigestyakh,62.0989,126.6992,-50.01,76,67,1.19,1612032600,RU


In [5]:
gmaps.configure(api_key=gkey)

In [36]:
locations = CityDataFrame[["Lat", "Lng"]].astype(float) 
Humidity = CityDataFrame['Humidity'].astype(float) 
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
IdealWeather = CityDataFrame.loc[CityDataFrame["MaxTemp"] < 80 ] 
IdealWeather =  CityDataFrame.loc[CityDataFrame["MaxTemp"] > 70 ]  
IdealWeather = CityDataFrame.loc[CityDataFrame["WindSpeed"] < 10]   
IdealWeather = CityDataFrame.loc[CityDataFrame["Cloudiness"]  == 0]    
IdealWeather = IdealWeather.drop(columns = {'Unnamed: 0'}) 
IdealWeather.dtypes 
IdealWeather = IdealWeather.dropna() 
IdealWeather

,city name,Lat,Lng,MaxTemp,Humidity,Cloudiness,WindSpeed,Date,country
5,port elizabeth,-33.9180,25.5701,66.20,88,0,5.75,1612032460,ZA
10,port lincoln,-34.7333,135.8667,59.54,81,0,12.82,1612032601,AU
13,puerto ayora,-0.7393,-90.3518,86.00,62,0,14.97,1612032602,EC
15,marzuq,14.4000,46.4667,62.62,60,0,3.65,1612032602,YE
16,nizwa,22.9333,57.5333,60.87,42,0,2.10,1612032545,OM
...,...,...,...,...,...,...,...,...,...
533,abu dhabi,24.4667,54.3667,71.60,64,0,2.30,1612032646,AE
546,yanji,42.9075,129.5078,-0.45,86,0,1.61,1612032703,CN
550,mecca,21.4267,39.8261,69.51,40,0,7.74,1612032481,SA
567,serici,44.4653,18.5581,39.20,81,0,3.44,1612032707,BA


In [41]:
locations = IdealWeather[["Lat", "Lng"]].astype(float)  
Humidity = IdealWeather['Humidity'].astype(float) 

In [42]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [53]:
# geocoordinates
target_coordinates = IdealWeather[["Lat", "Lng"]]
target_search = "Hotel"
target_radius = 5000
target_type = "place"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [54]:
response

<Response [200]>

In [55]:
places_data = response.json()

# Print the json (pretty printed)
print(places_data)

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}


In [52]:
IdealWeather[["Lat", "Lng"]]


,Lat,Lng
5,-33.9180,25.5701
10,-34.7333,135.8667
13,-0.7393,-90.3518
15,14.4000,46.4667
16,22.9333,57.5333
...,...,...
533,24.4667,54.3667
546,42.9075,129.5078
550,21.4267,39.8261
567,44.4653,18.5581


In [58]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in IdealWeather.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        IdealWeather.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("oops")

oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops


In [59]:
IdealWeather

,city name,Lat,Lng,MaxTemp,Humidity,Cloudiness,WindSpeed,Date,country,Hotel Name
5,port elizabeth,-33.9180,25.5701,66.20,88,0,5.75,1612032460,ZA,Island Way Villa
10,port lincoln,-34.7333,135.8667,59.54,81,0,12.82,1612032601,AU,Port Lincoln Hotel
13,puerto ayora,-0.7393,-90.3518,86.00,62,0,14.97,1612032602,EC,Angermeyer Waterfront Inn
15,marzuq,14.4000,46.4667,62.62,60,0,3.65,1612032602,YE,NaN
16,nizwa,22.9333,57.5333,60.87,42,0,2.10,1612032545,OM,Albayan Farm House
...,...,...,...,...,...,...,...,...,...,...
533,abu dhabi,24.4667,54.3667,71.60,64,0,2.30,1612032646,AE,Emirates Palace
546,yanji,42.9075,129.5078,-0.45,86,0,1.61,1612032703,CN,Yanbian Hotel
550,mecca,21.4267,39.8261,69.51,40,0,7.74,1612032481,SA,Pullman Zamzam Makkah
567,serici,44.4653,18.5581,39.20,81,0,3.44,1612032707,BA,Hotel Senad od Bosne


In [ ]:
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Bank amount: {bank}" for index, row in IdealWeather.iterrows()]
)


fig = gmaps.figure()
fig.add_layer(bank_layer)

fig